# Numerical feature generation

With this module is possible to create thousands of features based on the original ones. The majority of the code used was taken from the [autofeat](https://github.com/cod3licious/autofeat) library, which is a automated feature engineering tool. What I did was some minor changes in order to incorporate fit and transform capabilities, and in this way one can use this module together with sklearn pipelines.

The code already does some feature selection based on the correlations between features, so in order to prevent data leakage the procedure is done only in the training set through the fit method and then, with the transform method just the remaining features will be selected from the validation or test sets.

The code can be found [here](https://github.com/abreukuse/ml_utilities/blob/master/numerical_feature_engineering.py) and the original one from autofeat is [here](https://github.com/cod3licious/autofeat/blob/master/autofeat/feateng.py).

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', 3000)

from numerical_feature_engineering import NumericalFeatureEngineering

import os
os.environ['KAGGLE_USERNAME'] = 'kaggle_username'
os.environ['KAGGLE_KEY'] = 'kaggle_api_key'

In [2]:
!kaggle datasets download -d jsphyg/weather-dataset-rattle-package
!unzip 'weather-dataset-rattle-package.zip'

weather-dataset-rattle-package.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  weather-dataset-rattle-package.zip
replace weatherAUS.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: weatherAUS.csv          


In [3]:
data = pd.read_csv('weatherAUS.csv')
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [4]:
X = data.select_dtypes('number')
X.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
0,13.4,22.9,0.6,NaN,NaN,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8
1,7.4,25.1,0.0,NaN,NaN,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3
2,12.9,25.7,0.0,NaN,NaN,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2
3,9.2,28.0,0.0,NaN,NaN,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5
4,17.5,32.3,1.0,NaN,NaN,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7


In [5]:
# replace zero for 0.001 in order to apply log transformation.
X = pd.DataFrame(np.where(X==0, 0.001, X), columns=X.columns)
X.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
0,13.4,22.9,0.600,NaN,NaN,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8
1,7.4,25.1,0.001,NaN,NaN,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3
2,12.9,25.7,0.001,NaN,NaN,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2
3,9.2,28.0,0.001,NaN,NaN,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5
4,17.5,32.3,1.000,NaN,NaN,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7


In [6]:
y = data.RainTomorrow

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=15)

In [8]:
# Simple imputation
imputer = SimpleImputer()
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns = X_test.columns)

In [9]:
feature_engineering = NumericalFeatureEngineering(verbose=1)

In [10]:
feature_engineering.fit(X_train, correlation_threshold=0.8)

[feateng] Step 1: transformation of original features
[feateng] Generated 76 transformed features from 16 original features - done.
[feateng] Step 2: first combination of features
[feateng] Generated 4128 feature combinations from 4186 original feature tuples - done.
[feateng] Generated altogether 4207 new features in 2 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 1119 additional features
[feateng] Drop a total of 3 features ['Pressure3pm', 'Temp9am', 'Temp3pm'] from the original set


NumericalFeatureEngineering(max_steps=2, start_features=None,
                            transformations=('1/', 'exp', 'log', 'abs', 'sqrt',
                                             '^2', '^3'),
                            verbose=1)

In [11]:
X_train_engineered = feature_engineering.transform(X_train)
X_test_engineered = feature_engineering.transform(X_test)

# I still need to change the printed messages when the transform method is called

[feateng] Step 1: transformation of original features
[feateng] Generated 76 transformed features from 16 original features - done.
[feateng] Step 2: first combination of features
[feateng] Generated 4128 feature combinations from 4186 original feature tuples - done.
[feateng] Generated altogether 4207 new features in 2 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 4204 additional features
[feateng] Drop a total of 0 features [] from the original set


In [12]:
X_train_engineered.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Cloud9am,Cloud3pm,1/MinTemp,1/MaxTemp,1/Rainfall,log(Rainfall),Rainfall**2,Rainfall**3,1/Evaporation,log(Evaporation),Evaporation**2,Evaporation**3,1/Sunshine,log(Sunshine),1/WindSpeed9am,log(WindSpeed9am),WindSpeed9am**3,1/WindSpeed3pm,log(WindSpeed3pm),1/Humidity9am,1/Humidity3pm,1/Cloud9am,exp(Cloud9am),log(Cloud9am),1/Cloud3pm,exp(Cloud3pm),log(Cloud3pm),1/Temp9am,1/Temp3pm,MinTemp*Sunshine,MinTemp*WindSpeed9am,MinTemp*WindSpeed3pm,Humidity3pm*MinTemp,Cloud9am*MinTemp,Cloud3pm*MinTemp,MinTemp/MaxTemp,MinTemp/Rainfall,MinTemp*log(Rainfall),MinTemp*log(Sunshine),MinTemp*Sunshine**2,MinTemp*WindGustSpeed**2,MinTemp/WindSpeed9am,MinTemp*log(WindSpeed9am),Humidity9am**2*MinTemp,Humidity9am**3*MinTemp,Humidity3pm**2*MinTemp,Humidity3pm**3*MinTemp,Cloud9am**2*MinTemp,Cloud3pm**2*MinTemp,MinTemp/Temp3pm,MaxTemp*WindGustSpeed,Humidity9am*MaxTemp,Humidity3pm*MaxTemp,Cloud3pm*MaxTemp,MaxTemp/WindGustSpeed,MaxTemp*sqrt(WindSpeed9am),MaxTemp*log(WindSpeed3pm),MaxTemp*sqrt(WindSpeed3pm),Humidity9am**2*MaxTemp,sqrt(Humidity3pm)*MaxTemp,sqrt(Cloud9am)*MaxTemp,sqrt(Cloud3pm)*MaxTemp,Rainfall*Sunshine,Rainfall/MinTemp,Rainfall/MaxTemp,Rainfall/Evaporation,Evaporation**2*Rainfall,Rainfall/Sunshine,Rainfall*log(Sunshine),Rainfall*Sunshine**2,Rainfall*Sunshine**3,Rainfall*WindGustSpeed**2,Rainfall*WindGustSpeed**3,Rainfall/WindSpeed9am,Rainfall*log(WindSpeed9am),Rainfall*WindSpeed9am**2,Rainfall*WindSpeed9am**3,Rainfall/WindSpeed3pm,Rainfall*WindSpeed3pm**2,Rainfall*WindSpeed3pm**3,Rainfall/Cloud9am,Rainfall*exp(Cloud9am),Rainfall/Cloud3pm,Rainfall*exp(Cloud3pm),Rainfall/Temp9am,Rainfall/Temp3pm,Evaporation*WindSpeed9am,Evaporation*WindSpeed3pm,Evaporation*Humidity9am,Evaporation*Humidity3pm,Cloud9am*Evaporation,Cloud3pm*Evaporation,Evaporation*MinTemp**2,Evaporation*MaxTemp**3,Evaporation/Rainfall,Evaporation*log(Rainfall),Evaporation*sqrt(Rainfall),Evaporation/Sunshine,Evaporation*log(Sunshine),Evaporation*Sunshine**2,Evaporation/WindGustSpeed,Evaporation*WindGustSpeed**2,Evaporation*WindGustSpeed**3,Evaporation*log(WindSpeed9am),Evaporation*WindSpeed9am**2,Evaporation*WindSpeed3pm**2,Evaporation*Humidity9am**2,Evaporation*Humidity9am**3,Evaporation*Humidity3pm**2,sqrt(Cloud9am)*Evaporation,Cloud9am**2*Evaporation,Cloud9am**3*Evaporation,sqrt(Cloud3pm)*Evaporation,Cloud3pm**2*Evaporation,Cloud3pm**3*Evaporation,Evaporation*Temp9am**2,Sunshine*WindGustSpeed,Sunshine*WindSpeed3pm,Humidity9am*Sunshine,Humidity3pm*Sunshine,Cloud9am*Sunshine,Cloud3pm*Sunshine,Sunshine*Temp9am,Sunshine*Abs(MinTemp),Rainfall**2*Sunshine,Rainfall**3*Sunshine,Sunshine*log(Evaporation),Sunshine/WindGustSpeed,Sunshine*sqrt(WindSpeed9am),Sunshine*log(WindSpeed3pm),Humidity9am**2*Sunshine,Humidity9am**3*Sunshine,sqrt(Humidity3pm)*Sunshine,Humidity3pm**2*Sunshine,Humidity3pm**3*Sunshine,sqrt(Cloud9am)*Sunshine,Cloud9am**2*Sunshine,Cloud9am**3*Sunshine,sqrt(Cloud3pm)*Sunshine,Cloud3pm**2*Sunshine,Cloud3pm**3*Sunshine,Sunshine*Abs(Temp9am),Humidity9am*WindGustSpeed,Humidity3pm*WindGustSpeed,Cloud3pm*WindGustSpeed,Temp9am*WindGustSpeed,Temp3pm*WindGustSpeed,WindGustSpeed*Abs(MaxTemp),sqrt(Evaporation)*WindGustSpeed,sqrt(Sunshine)*WindGustSpeed,Humidity9am**2*WindGustSpeed,sqrt(Humidity3pm)*WindGustSpeed,sqrt(Cloud9am)*WindGustSpeed,sqrt(Cloud3pm)*WindGustSpeed,WindGustSpeed*Abs(Temp9am),WindGustSpeed*Abs(Temp3pm),Humidity3pm*WindSpeed9am,Cloud9am*WindSpeed9am,Cloud3pm*WindSpeed9am,WindSpeed9am/MinTemp,WindSpeed9am*Abs(MinTemp),MinTemp**2*WindSpeed9am,MaxTemp**2*WindSpeed9am,MaxTemp**3*WindSpeed9am,WindSpeed9am/Rainfall,WindSpeed9am*log(Rainfall),WindSpeed9am/Evaporation,WindSpeed9am*log(Sunshine),Sunshine**2*WindSpeed9am,Sunshine**3*WindSpeed9am,WindSpeed9am/WindSpeed3pm,Humidity9am**2*WindSpeed9am,Humidity9am**3*WindSpeed9am,Humidity3pm**2*WindSpeed9am,Humidity3pm**3*WindSpeed9am,Cloud9am**2*WindSpeed9am,Cloud9am**3*WindSpeed9am,Cloud3pm**2*WindSpeed9am

In [13]:
X_test_engineered.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Cloud9am,Cloud3pm,1/MinTemp,1/MaxTemp,1/Rainfall,log(Rainfall),Rainfall**2,Rainfall**3,1/Evaporation,log(Evaporation),Evaporation**2,Evaporation**3,1/Sunshine,log(Sunshine),1/WindSpeed9am,log(WindSpeed9am),WindSpeed9am**3,1/WindSpeed3pm,log(WindSpeed3pm),1/Humidity9am,1/Humidity3pm,1/Cloud9am,exp(Cloud9am),log(Cloud9am),1/Cloud3pm,exp(Cloud3pm),log(Cloud3pm),1/Temp9am,1/Temp3pm,MinTemp*Sunshine,MinTemp*WindSpeed9am,MinTemp*WindSpeed3pm,Humidity3pm*MinTemp,Cloud9am*MinTemp,Cloud3pm*MinTemp,MinTemp/MaxTemp,MinTemp/Rainfall,MinTemp*log(Rainfall),MinTemp*log(Sunshine),MinTemp*Sunshine**2,MinTemp*WindGustSpeed**2,MinTemp/WindSpeed9am,MinTemp*log(WindSpeed9am),Humidity9am**2*MinTemp,Humidity9am**3*MinTemp,Humidity3pm**2*MinTemp,Humidity3pm**3*MinTemp,Cloud9am**2*MinTemp,Cloud3pm**2*MinTemp,MinTemp/Temp3pm,MaxTemp*WindGustSpeed,Humidity9am*MaxTemp,Humidity3pm*MaxTemp,Cloud3pm*MaxTemp,MaxTemp/WindGustSpeed,MaxTemp*sqrt(WindSpeed9am),MaxTemp*log(WindSpeed3pm),MaxTemp*sqrt(WindSpeed3pm),Humidity9am**2*MaxTemp,sqrt(Humidity3pm)*MaxTemp,sqrt(Cloud9am)*MaxTemp,sqrt(Cloud3pm)*MaxTemp,Rainfall*Sunshine,Rainfall/MinTemp,Rainfall/MaxTemp,Rainfall/Evaporation,Evaporation**2*Rainfall,Rainfall/Sunshine,Rainfall*log(Sunshine),Rainfall*Sunshine**2,Rainfall*Sunshine**3,Rainfall*WindGustSpeed**2,Rainfall*WindGustSpeed**3,Rainfall/WindSpeed9am,Rainfall*log(WindSpeed9am),Rainfall*WindSpeed9am**2,Rainfall*WindSpeed9am**3,Rainfall/WindSpeed3pm,Rainfall*WindSpeed3pm**2,Rainfall*WindSpeed3pm**3,Rainfall/Cloud9am,Rainfall*exp(Cloud9am),Rainfall/Cloud3pm,Rainfall*exp(Cloud3pm),Rainfall/Temp9am,Rainfall/Temp3pm,Evaporation*WindSpeed9am,Evaporation*WindSpeed3pm,Evaporation*Humidity9am,Evaporation*Humidity3pm,Cloud9am*Evaporation,Cloud3pm*Evaporation,Evaporation*MinTemp**2,Evaporation*MaxTemp**3,Evaporation/Rainfall,Evaporation*log(Rainfall),Evaporation*sqrt(Rainfall),Evaporation/Sunshine,Evaporation*log(Sunshine),Evaporation*Sunshine**2,Evaporation/WindGustSpeed,Evaporation*WindGustSpeed**2,Evaporation*WindGustSpeed**3,Evaporation*log(WindSpeed9am),Evaporation*WindSpeed9am**2,Evaporation*WindSpeed3pm**2,Evaporation*Humidity9am**2,Evaporation*Humidity9am**3,Evaporation*Humidity3pm**2,sqrt(Cloud9am)*Evaporation,Cloud9am**2*Evaporation,Cloud9am**3*Evaporation,sqrt(Cloud3pm)*Evaporation,Cloud3pm**2*Evaporation,Cloud3pm**3*Evaporation,Evaporation*Temp9am**2,Sunshine*WindGustSpeed,Sunshine*WindSpeed3pm,Humidity9am*Sunshine,Humidity3pm*Sunshine,Cloud9am*Sunshine,Cloud3pm*Sunshine,Sunshine*Temp9am,Sunshine*Abs(MinTemp),Rainfall**2*Sunshine,Rainfall**3*Sunshine,Sunshine*log(Evaporation),Sunshine/WindGustSpeed,Sunshine*sqrt(WindSpeed9am),Sunshine*log(WindSpeed3pm),Humidity9am**2*Sunshine,Humidity9am**3*Sunshine,sqrt(Humidity3pm)*Sunshine,Humidity3pm**2*Sunshine,Humidity3pm**3*Sunshine,sqrt(Cloud9am)*Sunshine,Cloud9am**2*Sunshine,Cloud9am**3*Sunshine,sqrt(Cloud3pm)*Sunshine,Cloud3pm**2*Sunshine,Cloud3pm**3*Sunshine,Sunshine*Abs(Temp9am),Humidity9am*WindGustSpeed,Humidity3pm*WindGustSpeed,Cloud3pm*WindGustSpeed,Temp9am*WindGustSpeed,Temp3pm*WindGustSpeed,WindGustSpeed*Abs(MaxTemp),sqrt(Evaporation)*WindGustSpeed,sqrt(Sunshine)*WindGustSpeed,Humidity9am**2*WindGustSpeed,sqrt(Humidity3pm)*WindGustSpeed,sqrt(Cloud9am)*WindGustSpeed,sqrt(Cloud3pm)*WindGustSpeed,WindGustSpeed*Abs(Temp9am),WindGustSpeed*Abs(Temp3pm),Humidity3pm*WindSpeed9am,Cloud9am*WindSpeed9am,Cloud3pm*WindSpeed9am,WindSpeed9am/MinTemp,WindSpeed9am*Abs(MinTemp),MinTemp**2*WindSpeed9am,MaxTemp**2*WindSpeed9am,MaxTemp**3*WindSpeed9am,WindSpeed9am/Rainfall,WindSpeed9am*log(Rainfall),WindSpeed9am/Evaporation,WindSpeed9am*log(Sunshine),Sunshine**2*WindSpeed9am,Sunshine**3*WindSpeed9am,WindSpeed9am/WindSpeed3pm,Humidity9am**2*WindSpeed9am,Humidity9am**3*WindSpeed9am,Humidity3pm**2*WindSpeed9am,Humidity3pm**3*WindSpeed9am,Cloud9am**2*WindSpeed9am,Cloud9am**3*WindSpeed9am,Cloud3pm**2*WindSpeed9am

In [14]:
# The columns are the same in both train and test sets as expected.
all(X_train_engineered.columns == X_test_engineered.columns)

True